# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.5.0'

### Importar o dataset

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head(20)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [4]:
dataset.Geography.value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [5]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [6]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X[:, 2])

[0 0 0 ... 0 1 0]


In [7]:
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Feature Scaling

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
np.shape(X_train)

(8000, 12)

In [11]:
X_train

array([[-1.01460667, -0.5698444 ,  1.74309049, ...,  0.64259497,
        -1.03227043,  1.10643166],
       [-1.01460667,  1.75486502, -0.57369368, ...,  0.64259497,
         0.9687384 , -0.74866447],
       [ 0.98560362, -0.5698444 , -0.57369368, ...,  0.64259497,
        -1.03227043,  1.48533467],
       ...,
       [ 0.98560362, -0.5698444 , -0.57369368, ...,  0.64259497,
        -1.03227043,  1.41231994],
       [-1.01460667, -0.5698444 ,  1.74309049, ...,  0.64259497,
         0.9687384 ,  0.84432121],
       [-1.01460667,  1.75486502, -0.57369368, ...,  0.64259497,
        -1.03227043,  0.32472465]])

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [12]:
ann = tf.keras.models.Sequential()

 ###  Adding the input layer and the first hidden layer

In [13]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [14]:
ann.add(tf.keras.layers.Dense(units=9, activation='tahn'))

ValueError: Unknown activation function: tahn. Please ensure this object is passed to the `custom_objects` argument. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

### Adding the output layer

In [15]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Compiling the ANN

In [16]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Part 3 - Training the ANN

### Training the ANN on the Training set

In [17]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 849us/step - loss: 0.5236 - accuracy: 0.7925
Epoch 2/100
250/250 [==============================] - 0s 817us/step - loss: 0.4756 - accuracy: 0.7965
Epoch 3/100
250/250 [==============================] - 0s 861us/step - loss: 0.4535 - accuracy: 0.8014
Epoch 4/100
250/250 [==============================] - 0s 889us/step - loss: 0.4408 - accuracy: 0.8056
Epoch 5/100
250/250 [==============================] - 0s 761us/step - loss: 0.4332 - accuracy: 0.8085
Epoch 6/100
250/250 [==============================] - 0s 785us/step - loss: 0.4280 - accuracy: 0.8102
Epoch 7/100
250/250 [==============================] - 0s 909us/step - loss: 0.4245 - accuracy: 0.8130
Epoch 8/100
250/250 [==============================] - 0s 845us/step - loss: 0.4219 - accuracy: 0.8159
Epoch 9/100
250/250 [==============================] - 0s 793us/step - loss: 0.4194 - accuracy: 0.8185
Epoch 10/100
250/250 [==============================] - 0s 769us/step - l

250/250 [==============================] - 0s 793us/step - loss: 0.3560 - accuracy: 0.8534
Epoch 80/100
250/250 [==============================] - 0s 833us/step - loss: 0.3557 - accuracy: 0.8518
Epoch 81/100
250/250 [==============================] - 0s 841us/step - loss: 0.3559 - accuracy: 0.8544
Epoch 82/100
250/250 [==============================] - 0s 757us/step - loss: 0.3556 - accuracy: 0.8529
Epoch 83/100
250/250 [==============================] - 0s 761us/step - loss: 0.3551 - accuracy: 0.8539
Epoch 84/100
250/250 [==============================] - 0s 881us/step - loss: 0.3549 - accuracy: 0.8543
Epoch 85/100
250/250 [==============================] - 0s 809us/step - loss: 0.3546 - accuracy: 0.8543
Epoch 86/100
250/250 [==============================] - 0s 777us/step - loss: 0.3546 - accuracy: 0.8539
Epoch 87/100
250/250 [==============================] - 0s 833us/step - loss: 0.3546 - accuracy: 0.8551
Epoch 88/100
250/250 [==============================] - 0s 825us/step - loss:

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [18]:
y_pred = ann.predict(X_test)
y_pred

array([[0.25989875],
       [0.43086678],
       [0.17509961],
       ...,
       [0.0997864 ],
       [0.13477743],
       [0.19357488]], dtype=float32)

In [19]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [20]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1535   60]
 [ 208  197]]


##  A Confusion Matrix Indica um erro de aproximadamente 13,4% e acerto de 86,6%

# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)